In [12]:
import pandas as pd
import numpy as np
import datetime as dt
#from datetime import timedelta

import matplotlib.pyplot as plt
import seaborn as sns

from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.linear_model import CoxPHSurvivalAnalysis
from lifelines import CoxPHFitter

from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.exceptions import FitFailedWarning
import sksurv.util
import warnings
from lifelines.utils import k_fold_cross_validation

from sklearn.linear_model import LassoCV

from sksurv.preprocessing import OneHotEncoder


df=pd.read_csv("../1_data/private/preprocessed_data_Vitalant.csv")


/tmp/ipykernel_1516541/2361976919.py:26: DtypeWarning: Columns (2,16,18,21,24,26,28,29,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("../1_data/private/preprocessed_data_Vitalant.csv")


In [13]:
#for cox fitter
df_cox2= df.loc[:,['time_to_return','CENSORED', 'DONATION_VOLUME_DRAWN', 'DONATION_PROCEDURE_TYPE', 'PHLEBOTOMY_GROUP',
       'DONATION_TYPE', 'DONOR_GENDER', 'RACE_ETHNICITY','DONOR_COUNTRY_OF_BIRTH', 'DONOR_EDUCATION_DESC',
       'DONOR_WEIGHT', 'DONOR_HEIGHT', 'DONOR_AGE_AT_DONATION', 'Fixed_mobile', 'fixed_mobile_pattern',  'OUTCOME_TYPE', 'first_time',
        'cum_lifetime_donations', 'unit_rbc_loss', 'rbc_loss_last_12_months','rbc_loss_last_24_months', 'days_since_last_rbc_loss', 'days_since_last_drbc_loss']]


X3=pd.get_dummies(df_cox2)
X3.fillna(0, inplace=True)
# add interactions
X3['first_time*fixed']=X3["first_time"]*X3['Fixed_mobile']
X3['first_time*hgb']=X3["first_time"]*X3['OUTCOME_TYPE_low hgb']
X3['hgb*fixed']=X3["OUTCOME_TYPE_low hgb"]*X3['Fixed_mobile']

y3= X3['CENSORED']

pd.set_option('display.max_columns', 48)
X3.head()


,time_to_return,CENSORED,DONATION_VOLUME_DRAWN,DONOR_WEIGHT,DONOR_HEIGHT,DONOR_AGE_AT_DONATION,Fixed_mobile,first_time,cum_lifetime_donations,unit_rbc_loss,rbc_loss_last_12_months,rbc_loss_last_24_months,days_since_last_rbc_loss,days_since_last_drbc_loss,DONATION_PROCEDURE_TYPE_Automated,DONATION_PROCEDURE_TYPE_ND,DONATION_PROCEDURE_TYPE_Other,DONATION_PROCEDURE_TYPE_Undefined,DONATION_PROCEDURE_TYPE_WB,PHLEBOTOMY_GROUP_RBCP,PHLEBOTOMY_GROUP_Whole Blood,DONATION_TYPE_Allogeneic,DONATION_TYPE_Directed,DONATION_TYPE_Therapeutic,...,DONOR_EDUCATION_DESC_BACHELOR'S DEGREE,"DONOR_EDUCATION_DESC_GRADUATE DEGREE, DOCTORATE","DONOR_EDUCATION_DESC_GRADUATE DEGREE, MASTER'S","DONOR_EDUCATION_DESC_GRADUATE DEGREE, OTHER","DONOR_EDUCATION_DESC_GRADUATE DEGREE, PROFESSIONAL",DONOR_EDUCATION_DESC_HIGH SCHOOL GRAD-DIPLOMA OR EQUIV,DONOR_EDUCATION_DESC_HIGH SCHOOL GRADUATE OR EQUIVALENT,DONOR_EDUCATION_DESC_MASTER'S OR PROFESSIONAL DEGREE,DONOR_EDUCATION_DESC_NOT AVAILABLE,DONOR_EDUCATION_DESC_REFUSED,DONOR_EDUCATION_DESC_SOME COLLEGE OR TECHNICAL SCHOOL,DONOR_EDUCATION_DESC_UNAVAILABLE,DONOR_EDUCATION_DESC_UNKNOWN,fixed_mobile_pattern_FF,fixed_mobile_pattern_FM,fixed_mobile_pattern_MF,fixed_mobile_pattern_MM,fixed_mobile_pattern_last,OUTCOME_TYPE_completed,OUTCOME_TYPE_low hgb,OUTCOME_TYPE_other deferral,first_time*fixed,first_time*hgb,hgb*fixed
0,1017.0,0,530.0,190.0,63.0,35.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0
1,87.0,1,530.0,200.0,72.0,67.0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
2,46.0,1,530.0,200.0,72.0,67.0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
3,28.0,1,530.0,200.0,72.0,67.0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
4,85.0,0,530.0,200.0,72.0,67.0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0


In [14]:

# Split the data into training and test sets
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.7, random_state=0)

# Set up LassoCV model to find optimal penalty
model = LassoCV(cv=5, random_state=0)
model.fit(X_train3, y_train3)

# Get optimal penalty value
penalty = model.alpha_

print(penalty)

# Set up CoxPHFitter model with Lasso penalty
cph = CoxPHFitter(penalizer=penalty)

# Use k-fold cross-validation to fit the model
kf = KFold(n_splits=5, shuffle=True, random_state=0)
scores = []
for train_idx, val_idx in kf.split(X_train3):
    X_train_fold, X_val_fold = X_train3.iloc[train_idx], X_train3.iloc[val_idx]
    y_train_fold, y_val_fold = y_train3.iloc[train_idx], y_train3.iloc[val_idx]
    
    cph.fit(X_train_fold, duration_col='time_to_return', event_col='CENSORED')
    CV_score = cph.score(X_val_fold, scoring_method='concordance_index')
    scores.append(CV_score)

# Print average cross-validation score
print('Average cross-validation score:', sum(scores)/len(scores))


# Fit the model on the entire training set
cph.fit(X_train3, duration_col='time_to_return', event_col='CENSORED')

# Print summary of the model
cph.print_summary()

0.18238757869188243


/home/huzbah/.local/lib/python3.10/site-packages/lifelines/utils/__init__.py:1102: ConvergenceWarning: Column(s) ['cum_lifetime_donations', 'unit_rbc_loss', 'rbc_loss_last_12_months', 'rbc_loss_last_24_months', 'days_since_last_rbc_loss', 'days_since_last_drbc_loss', 'DONATION_PROCEDURE_TYPE_ND', 'DONOR_GENDER_UNKNOWN', 'DONOR_COUNTRY_OF_BIRTH_AFGHANISTAN', 'DONOR_COUNTRY_OF_BIRTH_AFRICA', 'DONOR_COUNTRY_OF_BIRTH_ALBANIA', 'DONOR_COUNTRY_OF_BIRTH_ALGERIA', 'DONOR_COUNTRY_OF_BIRTH_AMERICAN SAMOA', 'DONOR_COUNTRY_OF_BIRTH_ANGOLA', 'DONOR_COUNTRY_OF_BIRTH_ANTARCTICA', 'DONOR_COUNTRY_OF_BIRTH_ANTIGUA & BARBUDA', 'DONOR_COUNTRY_OF_BIRTH_ANTILLES, NETHERLANDS', 'DONOR_COUNTRY_OF_BIRTH_ARMENIA', 'DONOR_COUNTRY_OF_BIRTH_ARUBA', 'DONOR_COUNTRY_OF_BIRTH_ASHMORE & CARTIER ISLANDS', 'DONOR_COUNTRY_OF_BIRTH_AUSTRIA', 'DONOR_COUNTRY_OF_BIRTH_AZERBAIJAN', 'DONOR_COUNTRY_OF_BIRTH_BAHAMAS, THE', 'DONOR_COUNTRY_OF_BIRTH_BAHRAIN', 'DONOR_COUNTRY_OF_BIRTH_BANGLADESH', 'DONOR_COUNTRY_OF_BIRTH_BARBADOS', 'D

ConvergenceError: delta contains nan value(s). Convergence halted. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-model

In [ ]:
from forestplot import forestplot
# Extract the coefficients and their confidence intervals

ci = cph.confidence_intervals_#.iloc[[2, 21, 22, 5, 4, 6, 23, 10, 16, 17, 14, 13, 11, 25, 19, 9, 24, 18, 15, 20, 1],:]

coef = cph.hazard_ratios_#[['Fixed_mobile', 'OUTCOME_TYPE_no draw', "OUTCOME_TYPE_other deferral",
                          # 'rbc_loss_last_12_months','unit_rbc_loss','rbc_loss_last_24_months', 
                           #'first_time*fixed',  'sex_M','race_Mixed Race',  'race_White', 
                           #'race_African Black', 'donation_product_x_WB', 'donation_product_x_DEF PROD',
                           #'hgb*fixed', 'OUTCOME_TYPE_completed', 'sex_F', 'first_time*hgb',
                           #'race_unknown','race_Asian', 'OUTCOME_TYPE_low hgb', 'first_time']]

covariate=coef.index

# Create a dataframe to store the coefficients and their confidence intervals
data = pd.DataFrame({'covariate': covariate, 'coef': coef, 'lower': ci['95% lower-bound'], 'upper': ci['95% upper-bound']})

data['lower'] = np.exp(data['lower'])
data['upper'] = np.exp(data['upper'])

# Sort the dataframe in descending order of coefficients
data = data.sort_values('coef', ascending=False)

# Create a numpy array of indices for each covariate
ind = np.arange(data.shape[0])


fp=forestplot(data,  # the dataframe with results data
              estimate="coef",  # col containing estimated effect size 
              ll="lower", hl="upper",  # columns containing conf. int. lower and higher limits
              varlabel="covariate",  # column containing variable label
              ylabel="Confidence interval",  # y-label title
              xlabel="Hazard Ratio",  # x-label title
             )
fp.axvline(x=1.00, linestyle='--', color='gray')
plt.show()